In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow
np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import tensorflow_addons as tfa
from sklearn.metrics import log_loss
import tensorflow as tf

data_train = pd.read_csv('train_features.csv')
data_test = pd.read_csv('test_features.csv')
data_train_target_ns = pd.read_csv('train_targets_nonscored.csv')
data_train_target_s = pd.read_csv('train_targets_scored.csv')
sub = pd.read_csv('sample_submission.csv')

In [2]:
def preprocess(df):
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({24: 0, 48: 1, 72:2})
    del df['sig_id']
    return df

train = preprocess(data_train)
test = preprocess(data_test)

del data_train_target_s['sig_id']

In [3]:
def create_model(num_columns):
    model = Sequential()
    model.add(Input(num_columns))
    model.add(BatchNormalization())
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(206, activation='sigmoid'))
    
    optimizer = tfa.optimizers.Lookahead('adam',sync_period=10)
    
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    
    model.summary()
    return model

In [4]:
def metric(y_true, y_pred):
    metrics = []
    for _target in data_train_target_s.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels=[0,1]))
    return np.mean(metrics)

In [5]:
N_STARTS = 2
tf.random.set_seed(42)

res = data_train_target_s.copy()
sub.loc[:, data_train_target_s.columns] = 0
sub.loc[:, data_train_target_s.columns] = 0

for seed in range(N_STARTS):
    for n, (train_idx, test_idx) in enumerate(KFold(n_splits=5, random_state=seed, shuffle=True).split(data_train_target_s, data_train_target_s)):
        print(f'Fold {n}')
    
        model = create_model(875)
        checkpoint_path = f'repeat:{seed}_Fold:{n}.h5'
        reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
        cb_checkpt = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 0, save_best_only = True,
                                     save_weights_only = True, mode = 'min')
        model.fit(train.values[train_idx],
                  data_train_target_s.values[train_idx],
                  validation_data=(train.values[test_idx], data_train_target_s.values[test_idx]),
                  epochs=25, batch_size=128,
                  callbacks=[reduce_lr_loss, cb_checkpt], verbose=1
                 )
        
        model.load_weights(checkpoint_path)
        test_predict = model.predict(test.values)
        val_predict = model.predict(train.values[test_idx])
        
        sub.loc[:, data_train_target_s.columns] += test_predict
        res.loc[test_idx, data_train_target_s.columns] += val_predict
        print('')
    
sub.loc[:, data_train_target_s.columns] /= ((n+1) * N_STARTS)
res.loc[:, data_train_target_s.columns] /= N_STARTS

Fold 0
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 875)               3500      
_________________________________________________________________
dense (Dense)                (None, 2048)              1794048   
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)         

Epoch 1/25
149/149 [==============================] - 19s 126ms/step - loss: 0.6865 - accuracy: 0.0131 - val_loss: 0.4211 - val_accuracy: 0.0046 - lr: 0.0010
Epoch 2/25
149/149 [==============================] - 18s 122ms/step - loss: 0.2566 - accuracy: 0.0122 - val_loss: 0.0961 - val_accuracy: 0.0260 - lr: 0.0010
Epoch 3/25
149/149 [==============================] - 19s 130ms/step - loss: 0.0709 - accuracy: 0.0161 - val_loss: 0.0409 - val_accuracy: 0.0187 - lr: 0.0010
Epoch 4/25
149/149 [==============================] - 18s 118ms/step - loss: 0.0374 - accuracy: 0.0210 - val_loss: 0.0279 - val_accuracy: 0.0353 - lr: 0.0010
Epoch 5/25
149/149 [==============================] - 19s 126ms/step - loss: 0.0292 - accuracy: 0.0266 - val_loss: 0.0242 - val_accuracy: 0.0328 - lr: 0.0010
Epoch 6/25
149/149 [==============================] - 18s 124ms/step - loss: 0.0253 - accuracy: 0.0293 - val_loss: 0.0222 - val_accuracy: 0.0275 - lr: 0.0010
Epoch 7/25
149/149 [==============================] 

Epoch 14/25
149/149 [==============================] - 19s 130ms/step - loss: 0.0200 - accuracy: 0.0366 - val_loss: 0.0190 - val_accuracy: 0.0332 - lr: 0.0010
Epoch 15/25
149/149 [==============================] - 19s 125ms/step - loss: 0.0199 - accuracy: 0.0361 - val_loss: 0.0189 - val_accuracy: 0.0281 - lr: 0.0010
Epoch 16/25
149/149 [==============================] - 19s 130ms/step - loss: 0.0195 - accuracy: 0.0362 - val_loss: 0.0187 - val_accuracy: 0.0380 - lr: 0.0010
Epoch 17/25
149/149 [==============================] - 19s 127ms/step - loss: 0.0194 - accuracy: 0.0393 - val_loss: 0.0185 - val_accuracy: 0.0437 - lr: 0.0010
Epoch 18/25
149/149 [==============================] - 19s 124ms/step - loss: 0.0195 - accuracy: 0.0402 - val_loss: 0.0184 - val_accuracy: 0.0409 - lr: 0.0010
Epoch 19/25
149/149 [==============================] - 19s 127ms/step - loss: 0.0192 - accuracy: 0.0431 - val_loss: 0.0183 - val_accuracy: 0.0537 - lr: 0.0010
Epoch 20/25
149/149 [=========================

Epoch 1/25
149/149 [==============================] - 20s 135ms/step - loss: 0.6857 - accuracy: 0.0145 - val_loss: 0.4056 - val_accuracy: 0.0202 - lr: 0.0010
Epoch 2/25
149/149 [==============================] - 23s 152ms/step - loss: 0.2494 - accuracy: 0.0157 - val_loss: 0.0887 - val_accuracy: 0.0290 - lr: 0.0010
Epoch 3/25
149/149 [==============================] - 25s 168ms/step - loss: 0.0671 - accuracy: 0.0202 - val_loss: 0.0383 - val_accuracy: 0.0386 - lr: 0.0010
Epoch 4/25
149/149 [==============================] - 24s 158ms/step - loss: 0.0372 - accuracy: 0.0213 - val_loss: 0.0277 - val_accuracy: 0.0290 - lr: 0.0010
Epoch 5/25
149/149 [==============================] - 21s 139ms/step - loss: 0.0280 - accuracy: 0.0246 - val_loss: 0.0240 - val_accuracy: 0.0393 - lr: 0.0010
Epoch 6/25
149/149 [==============================] - 19s 128ms/step - loss: 0.0247 - accuracy: 0.0267 - val_loss: 0.0225 - val_accuracy: 0.0489 - lr: 0.0010
Epoch 7/25
149/149 [==============================] 

Epoch 14/25
149/149 [==============================] - 18s 123ms/step - loss: 0.0199 - accuracy: 0.0376 - val_loss: 0.0191 - val_accuracy: 0.0393 - lr: 0.0010
Epoch 15/25
149/149 [==============================] - 19s 124ms/step - loss: 0.0198 - accuracy: 0.0380 - val_loss: 0.0190 - val_accuracy: 0.0426 - lr: 0.0010
Epoch 16/25
149/149 [==============================] - 18s 122ms/step - loss: 0.0196 - accuracy: 0.0371 - val_loss: 0.0189 - val_accuracy: 0.0399 - lr: 0.0010
Epoch 17/25
149/149 [==============================] - 18s 120ms/step - loss: 0.0195 - accuracy: 0.0397 - val_loss: 0.0187 - val_accuracy: 0.0468 - lr: 0.0010
Epoch 18/25
149/149 [==============================] - 18s 123ms/step - loss: 0.0195 - accuracy: 0.0425 - val_loss: 0.0186 - val_accuracy: 0.0563 - lr: 0.0010
Epoch 19/25
149/149 [==============================] - 19s 130ms/step - loss: 0.0194 - accuracy: 0.0451 - val_loss: 0.0185 - val_accuracy: 0.0493 - lr: 0.0010
Epoch 20/25
149/149 [=========================

Epoch 1/25
149/149 [==============================] - 23s 154ms/step - loss: 0.6852 - accuracy: 0.0141 - val_loss: 0.4065 - val_accuracy: 0.0025 - lr: 0.0010
Epoch 2/25
149/149 [==============================] - 24s 160ms/step - loss: 0.2489 - accuracy: 0.0161 - val_loss: 0.0897 - val_accuracy: 0.0422 - lr: 0.0010
Epoch 3/25
149/149 [==============================] - 22s 149ms/step - loss: 0.0680 - accuracy: 0.0204 - val_loss: 0.0389 - val_accuracy: 0.0302 - lr: 0.0010
Epoch 4/25
149/149 [==============================] - 26s 172ms/step - loss: 0.0373 - accuracy: 0.0231 - val_loss: 0.0282 - val_accuracy: 0.0166 - lr: 0.0010
Epoch 5/25
149/149 [==============================] - 21s 139ms/step - loss: 0.0281 - accuracy: 0.0260 - val_loss: 0.0239 - val_accuracy: 0.0223 - lr: 0.0010
Epoch 6/25
149/149 [==============================] - 20s 136ms/step - loss: 0.0247 - accuracy: 0.0281 - val_loss: 0.0221 - val_accuracy: 0.0365 - lr: 0.0010
Epoch 7/25
149/149 [==============================] 

Epoch 14/25
149/149 [==============================] - 24s 160ms/step - loss: 0.0201 - accuracy: 0.0380 - val_loss: 0.0195 - val_accuracy: 0.0630 - lr: 0.0010
Epoch 15/25
149/149 [==============================] - 23s 152ms/step - loss: 0.0197 - accuracy: 0.0423 - val_loss: 0.0193 - val_accuracy: 0.0399 - lr: 0.0010
Epoch 16/25
149/149 [==============================] - 23s 157ms/step - loss: 0.0195 - accuracy: 0.0456 - val_loss: 0.0191 - val_accuracy: 0.0710 - lr: 0.0010
Epoch 17/25
149/149 [==============================] - 20s 135ms/step - loss: 0.0195 - accuracy: 0.0486 - val_loss: 0.0189 - val_accuracy: 0.0567 - lr: 0.0010
Epoch 18/25
149/149 [==============================] - 25s 168ms/step - loss: 0.0192 - accuracy: 0.0499 - val_loss: 0.0188 - val_accuracy: 0.0649 - lr: 0.0010
Epoch 19/25
149/149 [==============================] - 24s 161ms/step - loss: 0.0193 - accuracy: 0.0523 - val_loss: 0.0186 - val_accuracy: 0.0703 - lr: 0.0010
Epoch 20/25
149/149 [=========================

In [6]:
print(f'OOF Metric: {metric(data_train_target_s, res)}')

OOF Metric: 0.004675162921113734


In [7]:
sub.loc[test['cp_type']==1, data_train_target_s.columns] = 0

In [8]:
sub.to_csv('submission.csv', index=False)